<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/dbdevrel/blob/main/multicloud-oracledb-at-azure/oracle-azure-similarity-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oracle Database Similarity Search on Azure

This notebook demonstrates how to connect to an Oracle Autonomous Database hosted on Microsoft Azure and perform semantic similarity searches on text or numerical data using vector embeddings and similarity metrics.

## Overview

- **Cloud Platform**: Microsoft Azure
- **Database**: Oracle Autonomous Database@Azure
- **Embedding Model**: sentence-transformers for semantic embeddings
- **Similarity Metric**: Cosine similarity and Euclidean distance
- **Use Cases**: Document retrieval, product recommendations, semantic search, vector similarity

## Prerequisites

- Azure subscription with Oracle Database@Azure access
- Oracle Autonomous Database instance running on Azure
- Database connection details (endpoint, credentials)
- Google Colab environment (or local Jupyter Notebook)

## Section 1: Import Required Libraries

In [ ]:
# Install required packages for Google Colab
import subprocess
import sys

packages = [
    'cx_Oracle>=8.0',
    'azure-identity',
    'azure-keyvault-secrets',
    'pandas',
    'numpy',
    'scikit-learn',
    'sentence-transformers',
    'matplotlib',
    'seaborn'
]

print("Installing required packages...")
for package in packages:
    print(f"  Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("\n✓ All packages installed successfully!")

In [ ]:
# Import Required Libraries
import cx_Oracle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer
import warnings
import json
from typing import Tuple, List, Optional

# Azure libraries
try:
    from azure.identity import DefaultAzureCredential, ClientSecretCredential
    from azure.keyvault.secrets import SecretClient
    AZURE_AVAILABLE = True
except ImportError:
    AZURE_AVAILABLE = False
    print("⚠ Azure libraries available but optional for local testing")

warnings.filterwarnings('ignore')

print("✓ Core libraries imported successfully!")
if AZURE_AVAILABLE:
    print("✓ Azure integration libraries loaded")

## Interactive Search Function

Use this cell to perform custom similarity searches on your data:

In [ ]:
# Comprehensive Results Visualization and Analysis
import matplotlib.pyplot as plt
import seaborn as sns

if search_engine is not None and len(all_results) > 0:
    # Configure plotting style
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (12, 8)

    # Create subplots for each query
    fig, axes = plt.subplots(1, len(all_results), figsize=(15, 5))
    if len(all_results) == 1:
        axes = [axes]

    for idx, result_set in enumerate(all_results):
        query = result_set['query']
        results = result_set['results']

        # Create bar chart for similarity scores
        colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(results)))
        bars = axes[idx].barh(range(len(results)), results['similarity_score'], color=colors)

        axes[idx].set_yticks(range(len(results)))
        axes[idx].set_yticklabels([f"Rank {i+1}" for i in range(len(results))], fontsize=9)
        axes[idx].set_xlabel('Similarity Score', fontsize=10)
        axes[idx].set_title(f'Query {idx+1}: "{query[:40]}..."', fontsize=11, fontweight='bold')
        axes[idx].set_xlim(0, 1)

        # Add value labels
        for i, (bar, score) in enumerate(zip(bars, results['similarity_score'])):
            axes[idx].text(score + 0.02, i, f'{score:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.show()
    print("✓ Visualization displayed above\n")

    # Summary statistics
    print("="*80)
    print("SEARCH ANALYTICS SUMMARY")
    print("="*80)
    print(f"Total documents indexed: {len(df)}")
    print(f"Embedding dimension: {df['embeddings'].iloc[0].shape[0]}")
    print(f"Total queries executed: {len(all_results)}")

    # Aggregate statistics
    all_scores = []
    for result_set in all_results:
        all_scores.extend(result_set['results']['similarity_score'].values)

    if all_scores:
        print(f"\nSimilarity Score Statistics:")
        print(f"  Minimum: {min(all_scores):.4f}")
        print(f"  Maximum: {max(all_scores):.4f}")
        print(f"  Mean: {np.mean(all_scores):.4f}")
        print(f"  Median: {np.median(all_scores):.4f}")
        print(f"  Std Dev: {np.std(all_scores):.4f}")

    print("="*80)

else:
    print("⚠ No results available for visualization")

## Section 2: Configure Azure Credentials and Database Parameters

In [ ]:
# Configure Azure Credentials and Database Connection
import os
from google.colab import userdata

# Method 1: Retrieve credentials from Google Colab Secrets
try:
    ORACLE_HOST = userdata.get('ORACLE_AZURE_HOST')
    ORACLE_USER = userdata.get('ORACLE_AZURE_USER')
    ORACLE_PASSWORD = userdata.get('ORACLE_AZURE_PASSWORD')
    ORACLE_SERVICE = userdata.get('ORACLE_AZURE_SERVICE')
    print("✓ Database credentials loaded from Colab Secrets")
except:
    # Method 2: Use environment variables or hardcoded values for testing
    ORACLE_HOST = os.getenv('ORACLE_AZURE_HOST', 'your-oracle-db.oraclecloud.com')
    ORACLE_USER = os.getenv('ORACLE_AZURE_USER', 'admin')
    ORACLE_PASSWORD = os.getenv('ORACLE_AZURE_PASSWORD', 'your_password_here')
    ORACLE_SERVICE = os.getenv('ORACLE_AZURE_SERVICE', 'ORCL')
    print("⚠ Using default/environment variables for credentials")

# Oracle Database Configuration
DB_CONFIG = {
    'host': ORACLE_HOST,
    'port': 1521,
    'service_name': ORACLE_SERVICE,
    'user': ORACLE_USER,
    'password': ORACLE_PASSWORD,
    'wallet_location': None  # Optional: path to wallet for mTLS
}

# Optional: Azure Key Vault Integration (for secure credential storage)
def get_credentials_from_azure_keyvault(vault_url: str, secret_names: dict) -> dict:
    """
    Retrieve database credentials from Azure Key Vault

    Args:
        vault_url: Azure Key Vault URL
        secret_names: Dict mapping config keys to secret names

    Returns:
        Updated configuration dictionary
    """
    if not AZURE_AVAILABLE:
        print("⚠ Azure libraries not available")
        return {}

    try:
        credential = DefaultAzureCredential()
        client = SecretClient(vault_url=vault_url, credential=credential)

        credentials = {}
        for key, secret_name in secret_names.items():
            credentials[key] = client.get_secret(secret_name).value

        print("✓ Credentials retrieved from Azure Key Vault")
        return credentials
    except Exception as e:
        print(f"⚠ Could not retrieve from Key Vault: {str(e)}")
        return {}

# Display configuration (without exposing password)
print("\n" + "="*60)
print("ORACLE DATABASE CONFIGURATION")
print("="*60)
print(f"Host: {DB_CONFIG['host']}")
print(f"Port: {DB_CONFIG['port']}")
print(f"Service Name: {DB_CONFIG['service_name']}")
print(f"Username: {DB_CONFIG['user']}")
print("="*60)

## Section 3: Connect to Oracle Database on Azure

In [ ]:
# Establish Oracle Database Connection
def create_oracle_connection(config: dict) -> Optional[cx_Oracle.Connection]:
    """
    Create a connection to Oracle Autonomous Database on Azure

    Args:
        config (dict): Database configuration

    Returns:
        cx_Oracle.Connection or None: Database connection object
    """
    try:
        # Build connection string
        if config.get('wallet_location'):
            # Using wallet for mTLS (recommended for production)
            dsn = cx_Oracle.makedsn(
                config['host'],
                config['port'],
                service_name=config['service_name']
            )
            connection = cx_Oracle.connect(
                user=config['user'],
                password=config['password'],
                dsn=dsn
            )
        else:
            # Direct connection string
            connection_string = (
                f"{config['user']}/{config['password']}@"
                f"{config['host']}:{config['port']}/{config['service_name']}"
            )
            connection = cx_Oracle.connect(connection_string)

        print("✓ Successfully connected to Oracle Autonomous Database on Azure")
        print(f"  Database Version: {connection.version}")
        return connection

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"✗ Database Connection Error: {error.message}")
        return None
    except Exception as e:
        print(f"✗ Connection Error: {str(e)}")
        print("\nTroubleshooting Tips:")
        print("  1. Verify database endpoint is correct")
        print("  2. Check network connectivity (firewall rules)")
        print("  3. Validate username and password")
        print("  4. Ensure Oracle Database@Azure is running")
        return None

# Establish connection
oracle_conn = create_oracle_connection(DB_CONFIG)

if oracle_conn:
    print("\n✓ Connection Status: ACTIVE")
else:
    print("\n✗ Connection Status: FAILED")
    print("Please update credentials and retry connection")

## Section 4: Load Data from Oracle Table

In [ ]:
# Load Data from Oracle Database
def load_data_from_oracle(connection: cx_Oracle.Connection,
                          table_name: str,
                          columns: Optional[List[str]] = None,
                          limit: Optional[int] = None) -> Optional[pd.DataFrame]:
    """
    Load data from an Oracle table into a pandas DataFrame

    Args:
        connection: cx_Oracle connection object
        table_name: Name of the table to query
        columns: List of column names (None = all)
        limit: Maximum number of rows to retrieve

    Returns:
        pd.DataFrame: Data loaded from the table
    """
    try:
        if columns:
            column_str = ", ".join(columns)
            query = f"SELECT {column_str} FROM {table_name}"
        else:
            query = f"SELECT * FROM {table_name}"

        if limit:
            query = f"SELECT * FROM ({query}) WHERE ROWNUM <= {limit}"

        # Read using pandas with Oracle connection
        df = pd.read_sql(query, connection)

        print(f"✓ Successfully loaded {len(df)} records from {table_name}")
        print(f"  Columns: {list(df.columns)}")
        print(f"  Data shape: {df.shape}")
        return df

    except Exception as e:
        print(f"✗ Error loading data: {str(e)}")
        return None

# Load sample data
if oracle_conn:
    # Example: Load from a sample documents table
    TABLE_NAME = 'DOCUMENTS'  # Replace with your actual table name
    COLUMNS = ['ID', 'TITLE', 'CONTENT', 'CATEGORY']  # Adjust columns as needed
    LIMIT = 100  # Limit to first 100 records for demo

    print("Loading data from Oracle Database...")
    df = load_data_from_oracle(oracle_conn, TABLE_NAME, COLUMNS, LIMIT)

    if df is not None and not df.empty:
        print("\n" + "="*60)
        print("DATA PREVIEW")
        print("="*60)
        print(df.head())
        print(f"\nData Info:")
        print(f"  Rows: {len(df)}")
        print(f"  Columns: {df.shape[1]}")
        print(f"  Memory Usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    else:
        print("⚠ No data loaded or connection failed")
else:
    print("Cannot load data: Database connection not established")

## Section 5: Prepare Data for Similarity Search

In [ ]:
# Load Embedding Model
print("Loading semantic embedding model...")
print("(First run may take 1-2 minutes as model is downloaded)")

try:
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("✓ Embedding model loaded successfully")
    print(f"  Model: all-MiniLM-L6-v2")
    print(f"  Embedding dimension: 384")
except Exception as e:
    print(f"✗ Error loading model: {str(e)}")
    embedding_model = None

# Function to generate embeddings
def generate_embeddings(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Generate semantic embeddings for text data

    Args:
        texts: List of text strings
        batch_size: Batch size for processing

    Returns:
        np.ndarray: Embeddings array (n_samples, 384)
    """
    print(f"Generating embeddings for {len(texts)} documents...")
    embeddings = embedding_model.encode(texts, show_progress_bar=True, batch_size=batch_size)
    print(f"✓ Generated embeddings with shape: {embeddings.shape}")
    return embeddings

# Prepare and embed data
if df is not None and embedding_model is not None and not df.empty:
    # Identify text column
    if 'CONTENT' in df.columns:
        text_column = 'CONTENT'
    elif 'TITLE' in df.columns:
        text_column = 'TITLE'
    else:
        # Use first text column found
        text_columns = df.select_dtypes(include=['object']).columns.tolist()
        text_column = text_columns[0] if text_columns else None

    if text_column:
        # Clean text data
        df['text_clean'] = (df[text_column].fillna('').astype(str).str.strip()
                           .str.replace('\n', ' ').str.replace('\r', ' '))

        # Remove empty documents
        df = df[df['text_clean'].str.len() > 0].reset_index(drop=True)

        print(f"\nPreparing text from column: '{text_column}'")
        print(f"Documents with text: {len(df)}")
        print(f"\nSample text:\n{df['text_clean'].iloc[0][:250]}...\n")

        # Generate embeddings
        embeddings = generate_embeddings(df['text_clean'].tolist())

        # Store embeddings
        df['embeddings'] = [embeddings[i] for i in range(len(df))]

        print(f"\n✓ Data preparation complete")
        print(f"  Total documents: {len(df)}")
        print(f"  Embedding dimension: {embeddings.shape[1]}")
    else:
        print("⚠ No text column found for embedding")
        df = None
else:
    print("⚠ Cannot prepare data: missing connection, model, or dataframe")

## Section 6: Implement Similarity Search Algorithm

In [ ]:
# Interactive similarity search function
def perform_custom_search(search_engine, query_text: str, top_k: int = 5, method: str = 'cosine'):
    """
    Perform custom similarity search with detailed results display

    Args:
        search_engine: OracleSimilaritySearch instance
        query_text: Query string
        top_k: Number of results to return
        method: 'cosine' or 'euclidean'
    """
    if search_engine is None:
        print("⚠ Search engine not available")
        return None

    print(f"\n🔍 SIMILARITY SEARCH")
    print(f"Query: '{query_text}'")
    print(f"Method: {method.upper()}")
    print(f"Top Results: {top_k}")
    print("-"*80)

    # Execute search
    if method == 'cosine':
        results = search_engine.cosine_similarity_search(query_text, top_k=top_k)
        score_col = 'similarity_score'
    else:
        results = search_engine.euclidean_distance_search(query_text, top_k=top_k)
        score_col = 'distance'

    # Display results
    for idx, row in results.iterrows():
        print(f"\n✓ Rank {row['rank']}: {row[score_col]:.4f}")

        # Show available metadata
        for col in results.columns:
            if col not in ['rank', 'similarity_score', 'distance', 'search_method']:
                value = row[col]
                if value is not None:
                    if isinstance(value, str) and len(str(value)) > 100:
                        print(f"  {col}: {str(value)[:100]}...")
                    else:
                        print(f"  {col}: {value}")

    print("\n" + "="*80)
    return results

# Example: Custom search
if search_engine is not None:
    # Modify this query to test
    custom_query = "Oracle database performance optimization on Azure"
    custom_results = perform_custom_search(
        search_engine,
        custom_query,
        top_k=5,
        method='cosine'
    )

## Section 7: Execute Similarity Search Query

In [ ]:
# Similarity Search Engine
class OracleSimilaritySearch:
    """
    Semantic similarity search engine for Oracle Autonomous Database documents
    """

    def __init__(self, dataframe: pd.DataFrame, embedding_model):
        """
        Initialize similarity search engine

        Args:
            dataframe: DataFrame with embeddings column
            embedding_model: Pre-trained embedding model
        """
        self.df = dataframe.copy()
        self.model = embedding_model
        self.embeddings = np.array([emb for emb in dataframe['embeddings'].values])
        print(f"✓ Search engine initialized with {len(self.df)} documents")

    def cosine_similarity_search(self, query: str, top_k: int = 5) -> pd.DataFrame:
        """
        Find most similar documents using cosine similarity

        Args:
            query: Query text
            top_k: Number of results to return

        Returns:
            pd.DataFrame: Top K results with similarity scores
        """
        # Generate query embedding
        query_embedding = self.model.encode([query])[0].reshape(1, -1)

        # Calculate cosine similarity
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]

        # Get top K results
        top_indices = np.argsort(similarities)[::-1][:top_k]

        # Build results dataframe
        results = self.df.iloc[top_indices].copy()
        results['similarity_score'] = similarities[top_indices]
        results['rank'] = range(1, len(results) + 1)
        results['search_method'] = 'cosine_similarity'

        # Clean up columns for display
        cols_to_drop = [col for col in results.columns if col in ['embeddings', 'text_clean']]
        if cols_to_drop:
            results = results.drop(columns=cols_to_drop)

        return results.reset_index(drop=True)

    def euclidean_distance_search(self, query: str, top_k: int = 5) -> pd.DataFrame:
        """
        Find most similar documents using Euclidean distance

        Args:
            query: Query text
            top_k: Number of results to return

        Returns:
            pd.DataFrame: Top K results with distance scores
        """
        # Generate query embedding
        query_embedding = self.model.encode([query])[0].reshape(1, -1)

        # Calculate Euclidean distance
        distances = euclidean_distances(query_embedding, self.embeddings)[0]

        # Get top K results (smallest distances)
        top_indices = np.argsort(distances)[:top_k]

        # Build results dataframe
        results = self.df.iloc[top_indices].copy()
        results['distance'] = distances[top_indices]
        results['rank'] = range(1, len(results) + 1)
        results['search_method'] = 'euclidean_distance'

        # Clean up columns
        cols_to_drop = [col for col in results.columns if col in ['embeddings', 'text_clean']]
        if cols_to_drop:
            results = results.drop(columns=cols_to_drop)

        return results.reset_index(drop=True)

    def hybrid_search(self, query: str, top_k: int = 5, method: str = 'cosine') -> pd.DataFrame:
        """
        Perform hybrid search combining multiple similarity metrics

        Args:
            query: Query text
            top_k: Number of results to return
            method: 'cosine' or 'euclidean'

        Returns:
            pd.DataFrame: Top K hybrid search results
        """
        if method == 'cosine':
            return self.cosine_similarity_search(query, top_k)
        else:
            return self.euclidean_distance_search(query, top_k)

# Initialize search engine
if df is not None and 'embeddings' in df.columns and embedding_model is not None:
    search_engine = OracleSimilaritySearch(df, embedding_model)
else:
    search_engine = None
    print("⚠ Cannot initialize search engine")

## Section 8: Display and Analyze Results

In [ ]:
# Execute Similarity Search Queries
if search_engine is not None:
    # Define example queries
    example_queries = [
        "artificial intelligence and machine learning",
        "cloud database solutions and Azure services",
        "data analytics and business intelligence"
    ]

    print("="*80)
    print("EXECUTING SIMILARITY SEARCH QUERIES")
    print("="*80)

    all_results = []

    for idx, query in enumerate(example_queries, 1):
        print(f"\n📋 Query {idx}: '{query}'")
        print("-"*80)

        # Perform cosine similarity search
        results = search_engine.cosine_similarity_search(query, top_k=3)
        all_results.append({
            'query': query,
            'results': results
        })

        # Display results
        for result_idx, row in results.iterrows():
            print(f"\n  Rank {row['rank']}: Score = {row['similarity_score']:.4f}")

            # Display relevant columns
            for col in results.columns:
                if col not in ['rank', 'similarity_score', 'search_method']:
                    value = row[col]
                    if isinstance(value, str) and len(value) > 80:
                        print(f"  {col}: {value[:80]}...")
                    else:
                        print(f"  {col}: {value}")

    print("\n" + "="*80)
else:
    print("⚠ Search engine not initialized. Cannot execute queries.")

## Cleanup and Best Practices

In [ ]:
# Close database connection
if oracle_conn:
    try:
        oracle_conn.close()
        print("✓ Oracle database connection closed")
    except Exception as e:
        print(f"⚠ Warning closing connection: {str(e)}")

print("\n" + "="*80)
print("BEST PRACTICES FOR PRODUCTION IMPLEMENTATION")
print("="*80)

best_practices = """
## Security Best Practices
✓ Use Azure Key Vault for credential management
✓ Enable mTLS (mutual TLS) with wallet files
✓ Implement firewall rules and network security groups
✓ Use managed identities instead of service principals
✓ Enable SQL audit logging and monitoring
✓ Apply row-level security (RLS) in Oracle

## Performance Optimization
✓ Cache embeddings in database (e.g., VECTOR datatype in Oracle 23c+)
✓ Use connection pooling (cx_Oracle.SessionPool)
✓ Batch process queries for large datasets
✓ Implement approximate nearest neighbor (ANN) search
✓ Create indexes on vector columns
✓ Monitor query execution time and optimize queries

## Scalability Considerations
✓ Use dedicated vector databases for large-scale applications:
  - Azure Cognitive Search
  - Pinecone
  - Milvus
  - Weaviate
✓ Implement pagination for large result sets
✓ Use async queries for non-blocking operations
✓ Implement rate limiting and caching layers

## Data Management
✓ Regularly update embeddings for new/modified documents
✓ Version embedding models (track which model created each embedding)
✓ Archive old embeddings periodically
✓ Maintain consistency between database and embedding cache
✓ Implement data validation and quality checks

## Monitoring & Logging
✓ Track query latency and response times
✓ Monitor database connection health
✓ Log embedding generation times
✓ Monitor similarity score distributions
✓ Set up alerts for performance degradation
✓ Use Azure Monitor for infrastructure metrics

## Testing & Validation
✓ Test with representative data samples
✓ Validate embedding quality and consistency
✓ Benchmark different similarity algorithms
✓ Test error scenarios (connection failures, timeouts)
✓ Load test with realistic query patterns
"""

print(best_practices)
print("="*80)

## References and Additional Resources

### Official Documentation
- [Oracle Autonomous Database on Azure](https://www.oracle.com/cloud/azure/)
- [cx_Oracle Python Library](https://cx-oracle.readthedocs.io/)
- [Sentence Transformers](https://www.sbert.net/)
- [Azure Oracle Database@Azure Docs](https://docs.oracle.com/en-us/iaas/Content/Database/Concepts/oracledatabaseathome.htm)

### Useful Tutorials
- Oracle Database@Azure Setup: [Oracle-Azure Documentation](https://github.com/oracle-devrel/oracle-autonomous-database-samples)
- Vector Search Implementation: [Semantic Search Guide](https://huggingface.co/blog/semantic-search-blog)

### Related Technologies
- Azure Cognitive Search for vector search at scale
- Oracle Vector Search (Oracle 23c, 26ai)
- Open-source vector databases: Milvus, Weaviate, Pinecone